In [ ]:
!pip install yt-dlp openai-whisper nltk torch gradio pydub

!sudo apt-get update
!sudo apt-get install -y ffmpeg nodejs

import os
import re
import json
import time
import torch
import nltk
import whisper
import yt_dlp
import gradio as gr
from datetime import timedelta

# --- Configuration and Initialization ---
# Ensure NLTK data is ready
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
except Exception:
    pass

class RobustVideoChunker:
    def __init__(self, model_size="base"):
        """
        Initialize the Whisper model.
        Use 'large-v3' or 'medium.en' for high-quality production results.
        'base' is used here for faster testing/demo purposes.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Loading Whisper model '{model_size}' on {self.device}...")
        self.model = whisper.load_model(model_size, device=self.device)
        print("Model loaded.")

    def download_audio(self, youtube_url, cookies_path):
        """
        Downloads audio using the cookies method to bypass bot protection.

        Args:
            youtube_url (str): The YouTube link.
            cookies_path (str): Path to the Netscape-formatted cookies file.
        """
        print(f"⬇️ Downloading audio from {youtube_url} using cookies from: {cookies_path}")

        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}],
            'outtmpl': 'temp_audio',
            'quiet': False,
            'nocheckcertificate': True,
            'cookiefile': cookies_path,
        }

        output_path = "audio.mp3"

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([youtube_url])

            # Rename output
            if os.path.exists("temp_audio.mp3"):
                if os.path.exists(output_path): os.remove(output_path)
                os.rename("temp_audio.mp3", output_path)
                return output_path
            return None

        except yt_dlp.DownloadError as e:
            if "Sign in to confirm you’re not a bot" in str(e):
                 print(f"\n❌ Download Error: Bot detection block. Please ensure the cookies file exists at '{cookies_path}' and is valid.")
            else:
                print(f"\n❌ Download Error: {e}")
            return None
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}")
            return None

    def transcribe_and_chunk(self, audio_path, max_duration=15.0):
        """
        Transcribes, aligns, and performs the two-pass semantic chunking.
        """
        print(" Transcribing audio (with word timestamps)...")
        result = self.model.transcribe(audio_path, word_timestamps=True)

        # --- 1. Alignment: Get Sentences with Timestamps ---
        print("⏱️ Aligning sentences to timestamps...")
        all_words = []
        for segment in result['segments']:
            if 'words' in segment:
                all_words.extend(segment['words'])
            else:
                 # Fallback for old Whisper versions or specific audio segments
                 print("Warning: Missing word-level timestamps in a segment.")

        sentences = nltk.sent_tokenize(result['text'])
        aligned_sentences = []
        word_idx = 0

        for sent in sentences:
            sent_obj = {"text": sent, "start": None, "end": None, "words": []}
            sent_clean = sent.replace(" ", "").lower()
            acc = ""

            while word_idx < len(all_words):
                w = all_words[word_idx]

                # Check if the word belongs to the current sentence
                if sent_obj["start"] is None: sent_obj["start"] = w["start"]
                sent_obj["end"] = w["end"]
                sent_obj["words"].append(w)

                acc += w["word"].replace(" ", "").lower()
                word_idx += 1

                # Heuristic break: when accumulated word characters match sentence length (with tolerance)
                if len(acc) >= len(sent_clean) * 0.9: break

            aligned_sentences.append(sent_obj)

        # --- 2. Chunking: Pass 1 (Aggregation) & Pass 2 (Enforcement) ---
        print("🧠 Performing semantic chunking...")
        chunks = []
        current_chunk = {"text": "", "start": None, "end": None, "sentences": [], "words": []}

        # PASS 1: AGGREGATION
        for sent in aligned_sentences:
            if sent["start"] is None: continue

            # If current chunk is empty, initialize it
            if not current_chunk["sentences"]:
                current_chunk["start"] = sent["start"]
                current_chunk["end"] = sent["end"]
                current_chunk["text"] = sent["text"]
                current_chunk["sentences"] = [sent]
                current_chunk["words"].extend(sent["words"])
                continue

            # Calculate theoretical new duration
            new_end = sent["end"]
            current_duration = new_end - current_chunk["start"]

            if current_duration <= max_duration:
                # Add to current chunk (Semantic Coherence)
                current_chunk["end"] = new_end
                current_chunk["text"] += " " + sent["text"]
                current_chunk["sentences"].append(sent)
                current_chunk["words"].extend(sent["words"])
            else:
                # Force Break (Time Constraint)
                chunks.append(current_chunk)

                # Start new chunk with the current sentence
                current_chunk = {
                    "start": sent["start"],
                    "end": sent["end"],
                    "text": sent["text"],
                    "sentences": [sent],
                    "words": sent["words"]
                }

        if current_chunk["start"] is not None:
            chunks.append(current_chunk)

        # PASS 2: ENFORCEMENT (Splitting Oversized Chunks)
        final_chunks = []
        chunk_counter = 1

        for chunk in chunks:
            duration = chunk["end"] - chunk["start"]

            if duration <= max_duration:
                final_chunks.append({
                    "chunk_id": chunk_counter,
                    "chunk_length": round(duration, 2),
                    "text": chunk["text"].strip(),
                    "start_time": round(chunk["start"], 2),
                    "end_time": round(chunk["end"], 2)
                })
                chunk_counter += 1
            else:
                # Oversized: Must split using word boundaries/pauses
                # Simple implementation: split at the word closest to max_duration
                sub_chunks = self._split_oversized_by_duration(chunk, max_duration, chunk_counter)
                final_chunks.extend(sub_chunks)
                chunk_counter += len(sub_chunks)

        return final_chunks

    def _split_oversized_by_duration(self, chunk, max_duration, start_id):
        """Splits oversized chunks using word timestamps to respect max_duration."""
        sub_chunks = []
        words = chunk["words"]
        current_start = words[0]["start"]
        sub_chunk_words = []

        for i, word in enumerate(words):
            sub_chunk_words.append(word)
            current_duration = word["end"] - current_start

            # Check if adding the NEXT word will exceed the limit
            next_word_end = words[i+1]["end"] if i+1 < len(words) else word["end"]
            next_duration = next_word_end - current_start

            # If next duration exceeds or this is the last word
            if next_duration > max_duration or i == len(words) - 1:
                # Finalize the current sub-chunk at the current word boundary

                # Use the last word's end time and the first word's start time
                end_time = sub_chunk_words[-1]["end"]

                sub_chunks.append({
                    "chunk_id": start_id + len(sub_chunks),
                    "chunk_length": round(end_time - current_start, 2),
                    "text": " ".join([w["word"] for w in sub_chunk_words]).strip(),
                    "start_time": round(current_start, 2),
                    "end_time": round(end_time, 2)
                })

                # Reset for the next sub-chunk
                if i < len(words) - 1:
                    current_start = words[i+1]["start"]
                    sub_chunk_words = []

        return sub_chunks


# --- MAIN EXECUTION AND GRADIO APP ---

def process_video(url, cookies_path="/content/cookies.txt"):
    """
    Main function to run the full pipeline.
    """
    if not os.path.exists(cookies_path):
        return json.dumps({"error": f"Cookies file not found at: {cookies_path}. Please create the file to bypass YouTube's sign-in requirement."}, indent=4)

    try:
        chunker = RobustVideoChunker(model_size="base") # Change to 'large-v3' for max quality

        # 1. Download (Cookies method)
        audio_file = chunker.download_audio(url, cookies_path)

        if not audio_file:
            return json.dumps({"error": "Download failed. Check console for details (e.g., cookie validity, file permissions)."}, indent=4)

        # 2. Transcribe and Chunk
        chunks = chunker.transcribe_and_chunk(audio_file)

        # Cleanup
        if os.path.exists(audio_file):
            os.remove(audio_file)

        return json.dumps(chunks, indent=4)

    except Exception as e:
        return json.dumps({"error": f"An unexpected processing error occurred: {str(e)}"}, indent=4)

# --- BONUS: Gradio App Interface ---
def launch_app():
    iface = gr.Interface(
        fn=process_video,
        inputs=[
            gr.Textbox(label="YouTube URL", placeholder="e.g., https://www.youtube.com/watch?v=jNQXAC9IVRw"),
            gr.Textbox(label="Cookies File Path", value="/content/cookies.txt", placeholder="e.g., my_cookies.txt")
        ],
        outputs=gr.JSON(label="Semantic Chunks (List of Dictionaries)"),
        title="YouTube Semantic Chunker with Cookies Fix",
        description="Extracts high-precision, semantically rich audio-text segments (max 15s) from any YouTube video. Requires a cookies file to bypass sign-in."
    )
    iface.launch(share=True)

if __name__ == "__main__":
    # --- Uncomment the line below to launch the Gradio web interface ---
    launch_app()

    # --- Or, run locally for testing (Make sure your youtube_cookies.txt is ready) ---
    #test_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw"
    #print(process_video(test_url))

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 kB in 1s (263 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem 

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 189MiB/s]


Model loaded.
⬇️ Downloading audio from https://www.youtube.com/watch?v=jNQXAC9IVRw using cookies from: /content/cookies.txt
[youtube] Extracting URL: https://www.youtube.com/watch?v=jNQXAC9IVRw
[youtube] jNQXAC9IVRw: Downloading webpage
[youtube] jNQXAC9IVRw: Downloading tv downgraded player API JSON
[youtube] jNQXAC9IVRw: Downloading web safari player API JSON


[youtube] jNQXAC9IVRw: Downloading m3u8 information
[info] jNQXAC9IVRw: Downloading 1 format(s): 92
[download] Sleeping 6.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 4
[download] Destination: temp_audio
[download] 100% of  621.28KiB in 00:00:00 at 2.02MiB/s                 
[FixupM3u8] Fixing MPEG-TS in MP4 container of "temp_audio"
[ExtractAudio] Destination: temp_audio.mp3
Deleting original file temp_audio (pass -k to keep)
 Transcribing audio (with word timestamps)...
⏱️ Aligning sentences to timestamps...
🧠 Performing semantic chunking...
[
    {
        "chunk_id": 1,
        "chunk_length": 12.48,
        "text": "Alright so here we are one of the elephants. The cool thing about these guys is that they have really, really, really long hunts. And that's cool.",
        "start_time": 1.1,
        "end_time": 13.58
    },
    {
        "chunk_id": 2,
        "chunk_length": 4.5,
        "text": "And that's pretty much all 

In [ ]:
test_url = "https://www.youtube.com/watch?v=1aA1WGON49E"
print(process_video(test_url))

Loading Whisper model 'base' on cuda...
Model loaded.
⬇️ Downloading audio from https://www.youtube.com/watch?v=1aA1WGON49E using cookies from: /content/cookies.txt
[youtube] Extracting URL: https://www.youtube.com/watch?v=1aA1WGON49E
[youtube] 1aA1WGON49E: Downloading webpage
[youtube] 1aA1WGON49E: Downloading tv downgraded player API JSON
[youtube] 1aA1WGON49E: Downloading web safari player API JSON


[youtube] 1aA1WGON49E: Downloading m3u8 information
[info] 1aA1WGON49E: Downloading 1 format(s): 96
[download] Sleeping 6.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 16
[download] Destination: temp_audio
[download] 100% of  155.91MiB in 00:00:00 at 261.95GiB/s
[FixupM3u8] Fixing MPEG-TS in MP4 container of "temp_audio"
[ExtractAudio] Destination: temp_audio.mp3
Deleting original file temp_audio (pass -k to keep)
 Transcribing audio (with word timestamps)...
⏱️ Aligning sentences to timestamps...
🧠 Performing semantic chunking...
[
    {
        "chunk_id": 1,
        "chunk_length": 5.64,
        "text": "Welcome to this course on mastering technical interviews for software engineering roles.",
        "start_time": 0.0,
        "end_time": 5.64
    },
    {
        "chunk_id": 2,
        "chunk_length": 11.02,
        "text": "These interviews often hinge on your ability to confidently solve problems using data structures an

In [ ]:
launch_app()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d95ea63ad24849b7b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
